# Übung 3

In [3]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

**Aufgabe 11**

**A:** Für den Datensatz in Unfaelle.txt wird folgendes logistisches Regressionsmodell
$$M : \text{Unfall} ∼ \text{Alter} + \text{Fahrpraxis} + \text{Beruf} + \text{Geschlecht}\text{ (1)}$$
betrachtet. Durch Ausdrücke wie (1) werden in R Regressionsmodelle
unterschiedlichster Art spezifiziert. Geben Sie die Modellgleichung von
M in mathematischer Notation an. Wie viele Parameter werden benötigt?
Hinweis: Die Variablen Beruf und Geschlecht müssen zunächst in Dummy-
Variablen umkodiert werden.

In unserem Datensatz gibt es 3 Berufe:
- Physiker
- Zahnarzt
- Biologe

Sowie die beiden Geschlechter
- Mann
- Frau

Demzufolge brauchen wir jeweils für den ersten Fall 2, und den zweiten Fall eine Dummy-Variable.
$$
\text{Unfall}_i = \beta_0 + \beta_1 \text{ Alter} + \beta_2 \text{ Fahrpraxis} + \beta_3D_{3i} + \beta_4D_{4i} + \beta_5D_{5i} +\epsilon_i
$$

Wobei

| Beruf | D3 | D4 |
| --- | --- | --- |
| Physiker | 1 | 0 |
| Zahnarzt | 0 | 1 |
| Biologe | 0 | 0 |

| Geschlecht | D5 |
| --- | --- |
| Mann | 0 |
| Frau | 1 |

eine mögliche Kodierung für die Variablen wäre.

In [4]:
accidents = pd.read_csv("Unfaelle.txt", sep=" ")

print(f"Datensatz:\n{accidents.head()}")

Datensatz:
   Lnr  Unfall Geschlecht     Beruf  Alter  Fahrpraxis
0    1       0       Mann  Physiker     31          12
1    2       0       Frau  Physiker     37          18
2    3       0       Frau  Physiker     38          18
3    4       0       Frau  Physiker     44          25
4    5       0       Frau  Physiker     52          33


In [10]:
model = sm.formula.ols(formula="Unfall ~ Alter + Fahrpraxis + C(Beruf) + C(Geschlecht)", data=accidents).fit()

print(f"Kodierung der Koeffizienten: {model.model.exog_names}")

Kodierung der Koeffizienten: ['Intercept', 'C(Beruf)[T.Physiker]', 'C(Beruf)[T.Zahnarzt]', 'C(Geschlecht)[T.Mann]', 'Alter', 'Fahrpraxis']


**B:** Berechnen Sie in R die Koeffizientenschätzer.

In [6]:
print(model.params)

Intercept                0.142179
C(Beruf)[T.Physiker]    -0.011840
C(Beruf)[T.Zahnarzt]     0.041440
C(Geschlecht)[T.Mann]   -0.026057
Alter                    0.005209
Fahrpraxis              -0.012767
dtype: float64


**C:** Das Modell M habe die Parameter $β_0, . . . , β_p$. Welche anschauliche Bedeutung haben die exponierten Parameter $exp(β_0), . . . , exp(β_p)$?

Werden die Parameter exponiert, so erhält man die Odds-Ratios oder relative Risiken.

**Für logistische Regressionen**: Exponentierte Koeffizienten zeigen das Odds-Ratio an, wenn eine Prädiktorvariable um 1 Einheit erhöht wird, während alle anderen Variablen konstant bleiben.

**Für andere Modelle**: Exponentierte Koeffizienten können als relative Risiken oder Hazard-Ratios interpretiert werden.

**Allgemeine Interpretation**:*
- Ein exponentierter Koeffizient > 1 zeigt eine positive Assoziation zwischen Prädiktor und Outcome an.
- Ein exponentierter Koeffizient < 1 zeigt eine negative Assoziation an.
- Ein exponentierter Koeffizient = 1 zeigt keine Assoziation an.

**Aufgabe 12**

Berechnen Sie aus dem Modell M (Aufgabe 11) das Odds und die Wahrscheinlichkeit
- eines 30-jährigen Zahnarztes mit 10 Jahren Fahrpraxis,
- einer 29-jährigen Zahnärztin mit 11 Jahren Fahrpraxis,
- einer 50-jährigen Biologin mit 30 Jahren Fahrpraxis,

einen Unfall zu verursachen.

In [23]:
A_data = pd.DataFrame({'Alter': [30], 'Fahrpraxis': [10], 'Beruf': ['Zahnarzt'], 'Geschlecht': ['Mann']})
B_data = pd.DataFrame({'Alter': [29], 'Fahrpraxis': [11], 'Beruf': ['Zahnarzt'], 'Geschlecht': ['Frau']})
C_data = pd.DataFrame({'Alter': [50], 'Fahrpraxis': [30], 'Beruf': ['Biologe'], 'Geschlecht': ['Frau']})

A_pred = model.predict(A_data)
B_pred = model.predict(B_data)
C_pred = model.predict(C_data)

print(A_pred[0])
print(B_pred[0])
print(C_pred[0])

0.1861562246990183
0.19423782326872494
0.01960963985728692
Scenario A: Probability of an accident occurring = 0    0.546405
dtype: float64
Scenario B: Probability of an accident occurring = 0    0.548407
dtype: float64
Scenario C: Probability of an accident occurring = 0    0.504902
dtype: float64


**Aufgabe 13**
Betrachten Sie das logistische Regressionsmodell
$$
L : \text{Unfall} ∼ \text{Alter}
$$

für die Daten in Unfaelle.txt.

**A:** Berechnen Sie mit der Funktion `glm`, `coef` und `vcov` die Parameterschätzer
und die Varianz-Kovarianz Matrix V des Modells L.

In [34]:
model_log = sm.formula.logit(formula="Unfall ~ Alter", data=accidents).fit()

print(model_log.params)

cov_matrix = model_log.cov_params()
print(cov_matrix)

Optimization terminated successfully.
         Current function value: 0.100093
         Iterations 10
Intercept    8.794495
Alter       -0.327340
dtype: float64
           Intercept     Alter
Intercept   0.485038 -0.014985
Alter      -0.014985  0.000473


**B:** Geben Sie eine Formel an, die log(Odds) in Abhängigkeit von Alter
berechnet.

$$
\log \Bigg ( \frac{p}{1-p} \Bigg ) = \beta_0 + \beta_1 \text{Alter}
$$

**C:** Berechnen Sie mit der Varianz-Kovarianz Matrix die Standardabweichung von log(Odds).

In [38]:
# ?